In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# mnistデータダウンロード
mnist = input_data.read_data_sets("mnist/", one_hot=True)
# mnistの基本情報
pixels = 28 * 28
nums = 10

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [8]:

x = tf.placeholder(tf.float32, shape=(None, pixels), name="x")
y_ = tf.placeholder(tf.float32, shape=(None, nums), name="y_")

In [9]:
# 重みを初期化
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="b_"+name)
    return W

In [10]:
# バイアスを初期化
def bias_variable(name, size):
    b_init = tf.constant(0.1, shape=[size])
    b = tf.Variable(b_init, name="b_" + name)
    return b

In [11]:
# 畳み込みを行う関数
def conv2d(x, W):
    return tf.nn.conv2d(x ,W, strides=[1,1,1,1], padding="SAME")

In [12]:
# プーリング層を行う(maxpooling法)
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [13]:
# 最初の畳み込み層
with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable('conv1', [5,5,1,32])
    b_conv1 = bias_variable('conv1', 32)
    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [14]:
# 最初のプーリング層
with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool(h_conv1)

In [15]:
# 二番目の畳み込み層
with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable('conv2', [5,5,32,64])
    b_conv2 = bias_variable('conv2', 64)
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

In [16]:
# 二番目のプーリング層
with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool(h_conv2)

In [17]:
# 全結合層
with tf.name_scope('fully_connected') as scope:
    n = 7 * 7* 64
    W_fc = weight_variable('fc', [n, 1024])
    b_fc = bias_variable('fc', 1024)
    h_pool2_flat = tf.reshape(h_pool2, [-1,n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc) + b_fc)

In [18]:
# ドロップアウト(過剰適合の排除)
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

In [19]:
# 読み出し層
with tf.name_scope('read_out') as scope:
    W_fc2 = weight_variable('fc2', [1024, 10])
    b_fc2 = bias_variable('fc2', 10)
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)

In [21]:
# モデルの学習
with tf.name_scope('loss') as scope:
    cross_entoropy = -tf.reduce_sum(y_ * tf.log(y_conv))

with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entoropy)

In [22]:
# モデルの評価
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))

In [23]:
# feed_dictの設定
def set_feed(images, labels, prob):
    return {x: images, y_: labels, keep_prob: prob}

In [24]:
# 実行
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    tw = tf.summary.FileWriter('log_dir', graph=sess.graph)
    test_fd = set_feed(mnist.test.images, mnist.test.labels, 1)
    for step in range(10000):
        batch = mnist.train.next_batch(50)
        fd = set_feed(batch[0], batch[1], 0.5)
        _, loss = sess.run([train_step, cross_entoropy], feed_dict=fd)
        if step % 100 == 0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step=", step, "loss=", loss, "acc=", acc)

    acc = sess.run(accuracy_step, feed_dict=test_fd)
    print("正解率= ", acc)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step= 0 loss= 745.063 acc= 0.1039
step= 100 loss= 54.69 acc= 0.8352
step= 200 loss= 24.3444 acc= 0.9014
step= 300 loss= 17.5552 acc= 0.9215
step= 400 loss= 14.4278 acc= 0.9367
step= 500 loss= 8.97777 acc= 0.9407
step= 600 loss= 14.7365 acc= 0.9462
step= 700 loss= 10.8366 acc= 0.9499
step= 800 loss= 22.4055 acc= 0.9556
step= 900 loss= 8.99015 acc= 0.9552
step= 1000 loss= 10.5417 acc= 0.9612
step= 1100 loss= 21.839 acc= 0.9636
step= 1200 loss= 4.8151 acc= 0.9652
step= 1300 loss= 8.00093 acc= 0.9672
step= 1400 loss= 9.5831 acc= 0.9677
step= 1500 loss= 6.06854 acc= 0.9711
step= 1600 loss= 12.0865 acc= 0.9705
step= 1700 loss= 1.10853 acc= 0.9715
step= 1800 loss= 4.51882 acc= 0.9734
step= 1900 loss= 7.18997 acc= 0.9752
step= 2000 loss= 14.3412 acc= 0.9747
step= 2100 loss= 9.45247 acc= 0.9747
step= 2200 loss= 7.14857 acc= 0.976
step= 2300 loss= 7.21232 acc= 0.9775
step= 2400 loss= 8.44379 acc= 0.9765
step= 2500 loss= 3.

KeyboardInterrupt: 